# 퍼셉트론 (Perceptron)

## 1. 정의

- Perceptron is an algorithm for supervised learning of **binary classifiers** 
  - The binary classifiers are functions that map an input $x$ (a vector of real numbers) to an output value $f(x)$
    - $f(x)={\begin{cases}1&{\text{if }}\mathbf {w}\cdot \mathbf {x}+ b>0\\0&{\text{otherwise}}\end{cases}}$
    - where 
      - $w$ is a vector of real-valued weights (=features)
      - $w\cdot x$ is the dot product $\sum _{i=1}^{m}w_{i}x_{i}$ where $m$ is the number of inputs to the perceptron and $b$ is the bias. 
        - The bias shifts the decision boundary away from the origin and does not depend on any input value.
- It is a type of linear classifier, i.e. a classification algorithm that makes its predictions based on a linear predictor function combining a set of weights with the feature vector.

## 2 퍼셉트론 기본 구현

In [2]:
import tensorflow as tf
import numpy as np

In [80]:
class Perceptron:
    def __init__(self):
        self.x = tf.placeholder(tf.float32, [1, 2])
        self.weight = tf.Variable(tf.zeros([2, 1]))  # [[0.]  [0.]] 
        self.bias = tf.Variable(tf.zeros([1]))    # [[0.]]
    
    def setVar(self, sess, w, b):
        op1 = tf.assign(self.weight, w)
        op2 = tf.assign(self.bias, b)
        sess.run(op1)
        sess.run(op2)
        
    def prediction(self, sess, x):
        y = tf.add(tf.matmul(self.x, self.weight), self.bias)
        classification = tf.greater_equal(y, [0.0])
        result = sess.run(classification, feed_dict={self.x: x})
        if result[0]:
            return 1
        else:
            return 0

In [81]:
# AND Gate
# x1  x2  result
# --------------
#  0   0       0
#  1   0       0
#  0   1       0
#  1   1       1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    data = np.array([[[0.0, 0.0]], [[1.0, 0.0]], [[0.0, 1.0]], [[1.0, 1.0]]])
    and_gate = Perceptron()
    w = np.array([[0.5], [0.5]])
    b = np.array([-0.7])
    and_gate.setVar(sess, w, b)
    for x in data:
        result = and_gate.prediction(sess, x)
        print("{0}, {1} - {2}".format(x[0][0], x[0][1], result))

0.0, 0.0 - 0
1.0, 0.0 - 0
0.0, 1.0 - 0
1.0, 1.0 - 1


In [83]:
# OR Gate
# x1  x2  result
# --------------
#  0   0       0
#  1   0       1
#  0   1       1
#  1   1       1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    data = np.array([[[0.0, 0.0]], [[1.0, 0.0]], [[0.0, 1.0]], [[1.0, 1.0]]])
    or_gate = Perceptron()
    w = np.array([[0.5], [0.5]])
    b = np.array([-0.2])
    or_gate.setVar(sess, w, b)
    for x in data:
        result = or_gate.prediction(sess, x)
        print("{0}, {1} - {2}".format(x[0][0], x[0][1], result))

0.0, 0.0 - 0
1.0, 0.0 - 1
0.0, 1.0 - 1
1.0, 1.0 - 1


## 3 퍼셉트론 훈련

1. Initialize the weights and the bias
  1. Weights and bias may be initialized to 0 or to a small random value.
  
2. For each example $j$ in our training set $D$, perform the following steps over the input ${x}_{j}$ and desired output $d_{j}$:
  1. Calculate the predictive output:
    1. $y_{j}(t)=f[\mathbf {w} (t)\cdot \mathbf {x} _{j} + {b}(t)]$
  2. Update the weights:
    1. ${\mathbf {w}(t+1)=\mathbf {w}(t)+(d_{j}-y_{j}(t)) \cdot \mathbf {x}_{j}}$
    2. ${{b}(t+1)={b}(t)+(d_{j}-y_{j}(t))}$

3. The step 2 may be repeated until the iteration error ${\frac {1}{s}}\sum _{j=1}^{s}|d_{j}-y_{j}(t)|$ is less than a user-specified error threshold $\gamma$, or a predetermined number of iterations have been completed.

## 4 다층 퍼셉트론

In [84]:
class MultiPerceptron:
    def __init__(self):
        self.nand_gate = Perceptron()
        w = np.array([[-0.5], [-0.5]])
        b = np.array([0.7])
        self.nand_gate.setVar(sess, w, b)

        self.or_gate = Perceptron()
        w = np.array([[0.5], [0.5]])
        b = np.array([-0.2])
        self.or_gate.setVar(sess, w, b)

        self.and_gate = Perceptron()
        w = np.array([[0.5], [0.5]])
        b = np.array([-0.7])
        self.and_gate.setVar(sess, w, b)

    def prediction(self, sess, x):
        s1 = self.nand_gate.prediction(sess, x)
        s2 = self.or_gate.prediction(sess, x)
        y = self.and_gate.prediction(sess, [[s1, s2]])
        return y

In [85]:
# XOR Gate
# x1  x2  result
# --------------
#  0   0       0
#  1   0       1
#  0   1       1
#  1   1       0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    data = [[[0.0, 0.0]], [[1.0, 0.0]], [[0.0, 1.0]], [[1.0, 1.0]]]
    xor_gate = MultiPerceptron()
    for x in data:
        result = xor_gate.prediction(sess, x)
        print("{0}, {1} - {2}".format(x[0][0], x[0][1], result))

0.0, 0.0 - 0
1.0, 0.0 - 1
0.0, 1.0 - 1
1.0, 1.0 - 0
